# <center>Workflow for the CRC1333 project B07 - Technical Chemistry</center>
# <center>Experimental notebook</center>

---

This is the ``Experimental`` ``notebook``, where the actual analysis of the experiments takes place. It consists of three parts: ``Parsing``, ``analysis`` and ``DaRUS`` ``upload``. Within the scope of each project, multiple experiments are perfomed, hence multiple analyses are to be done. For each individual experiment this workflow is to be executed once, and the results can be appended to the project's dataset.

---

In [1]:
from sdRDM.generator import generate_python_api
from sdRDM import DataModel

In [2]:
# generate_python_api('specifications/datamodel_b07_tc.md', '', 'datamodel_b07_tc')

Import standard library python packages.

In [64]:
%reload_ext autoreload
%autoreload 2

from datamodel_b07_tc.tools import GCParser
from datamodel_b07_tc.tools import GstaticParser
from datamodel_b07_tc.tools import MFMParser
# from datamodel_b07_tc.tools import Calculator
from datamodel_b07_tc.tools import Calibrator
from datamodel_b07_tc.tools import get_volumetric_flow_mean
from datamodel_b07_tc.tools import get_initial_time_and_current
from datamodel_b07_tc.tools import assign_peaks
# from DEXPI2sdRDM import DEXPI2sdRDM

[autoreload of datamodel_b07_tc.tools.python_api.calibration failed: Traceback (most recent call last):
  File "/home/shambles/miniconda3/envs/b07/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/shambles/miniconda3/envs/b07/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/home/shambles/miniconda3/envs/b07/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/home/shambles/miniconda3/envs/b07/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    update_instances(old, new)
  File "/home/shambles/miniconda3/envs/b07/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 303, in update_instances
    ref.__class__ = new
  File "/mnt/c/Users/rscho/Documents/GitHub/software-driven-rdm/sdRDM/base/data

In [4]:
import os
import json
import ipywidgets as widgets
import logging
from logging import config
from IPython.display import display
from pathlib import Path

---
## Section 0: Path and Logging
---

Set up logger.

In [5]:
cwd = Path.cwd()
config_path = cwd / "datamodel_b07_tc/tools/logging/config.json"
print(config_path)

/mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/datamodel_b07_tc/tools/logging/config.json


In this section the data model and the dataset as well as all the output files necessary for analysis are parsed.  

Get path to the directory this file is located and check if it exists.

In [6]:
root = Path(os.path.abspath(''))
print("Path to this notebook's location:", root)
print('Is the path valid?', root.is_dir())

Path to this notebook's location: /mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc
Is the path valid? True


In [7]:
logging_config_path = root / "datamodel_b07_tc/tools/logging/config.json"
with open(logging_config_path) as logging_config_json:
    logging_config = json.load(logging_config_json)

In [8]:
logging.config.dictConfig(logging_config)

In [9]:
logger = logging.getLogger(__name__)
logger.debug("obacht")
logger.warning('uff')

2023-09-07 11:16:14,465 - __main__ - WARNING - uff


In [10]:
vars()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'from sdRDM.generator import generate_python_api\nfrom sdRDM import DataModel',
  "# generate_python_api('specifications/datamodel_b07_tc.md', '', 'datamodel_b07_tc')",
  "get_ipython().run_line_magic('reload_ext', 'autoreload')\nget_ipython().run_line_magic('autoreload', '2')\n\nfrom datamodel_b07_tc.tools import GCParser\nfrom datamodel_b07_tc.tools import GstaticParser\nfrom datamodel_b07_tc.tools import MFMParser\n# from datamodel_b07_tc.tools import Calculator\nfrom datamodel_b07_tc.tools import Calibrator\nfrom datamodel_b07_tc.tools import get_volumetric_flow_mean\nfrom datamodel_b07_tc.tools import get_initial_time_and_current\nfrom datamodel_b07_tc.tools import assign_peaks\n# from DEXPI2sdRDM import DE

---
## Section 1: Parsing
---

 Set path to datasets.

In [11]:
path_to_datasets = root / 'datasets'

List all available datasets in the directory.


In [12]:
files = path_to_datasets.iterdir()
json_files = {index:file for index, file in enumerate(files) if file.suffix == '.json'}
for index, file in json_files.items():
    print(f'{index}: {file.name}')

0: b07.json


Choose dataset to be loaded by its index.

In [13]:
index_dataset = 0
dataset, lib = DataModel.parse(json_files[index_dataset])

Visualize the data model.

In [14]:
# lib.Dataset.meta_tree()

Print current status of the dataset.

In [15]:
# print(dataset.json())

Set path to the directory containing the data.

In [16]:
path_data = root / 'data'

Path for the raw data.

In [17]:
# raw_data_path = Path('F:\Doktorand\\03_Messungen\Rohdaten')
path_raw_data = path_data / 'Rohdaten'

Instantiate an experiment object which holds all the information about one single experiment.

In [18]:
experiment = lib.Experiment()

### Potenstiostatic data

Provide name of the directory containing the potentiostatic measurement data.

In [19]:
path_echem = path_raw_data / '01_EChem'

Search in that directory for further subdirectories and print them.

In [20]:
subdirectories_echem = {index:directory for index, directory in enumerate(path_echem.iterdir())}
for index, directory in subdirectories_echem.items():
    print(f"{index}: {directory.name}")

0: 210728_ITO_TEST
1: CAD14-Cu@AB


Choose subdirectory by its index.

In [21]:
subdirectory_index_echem = 1
selected_subdirectory_echem = subdirectories_echem[subdirectory_index_echem]
print(selected_subdirectory_echem)

/mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/data/Rohdaten/01_EChem/CAD14-Cu@AB


Provide suffix of the file that contains the data.

In [22]:
suffix_echem = 'DTA'

Initialize the ``GstaticParser`` and print available files.

In [23]:
gstaticparser = GstaticParser(selected_subdirectory_echem, suffix_echem)
files_dict_echem = gstaticparser.available_files
for index, gstatic_file in files_dict_echem.items():
    print(f"{index}: {gstatic_file.stem}")

0: GSTATIC
1: POTDYN


Chose specific file.

In [24]:
file_index_echem = 0
file_echem = files_dict_echem[file_index_echem]
file_echem.name

'GSTATIC.DTA'

Extract the metadata from it using the ``GstaticParser`` and load into the data model.

In [25]:
gstatic_metadata_df, gstatic_metadata = gstaticparser.extract_metadata(file_index_echem)
potentiometric_measurement = lib.Measurement(measurement_type=lib.enums.MeasurementType.POTENTIOSTATIC, metadata=gstatic_metadata)
experiment.measurements = [potentiometric_measurement]
gstatic_metadata_df

,Parameter,Data_type,Value,Description
0,PSTAT,PSTAT,REF3000-19129,Potentiostat
1,IINIT,QUANT,-2.00000E+002,Initial I (mA/cm^2)
2,TINIT,QUANT,3.60000E+003,Initial Time (s)
3,IFINAL,QUANT,-2.00000E+002,Final I (mA/cm^2)
4,TFINAL,QUANT,0.00000E+000,Final Time (s)
5,SAMPLETIME,QUANT,1.00000E+000,Sample Period (s)
6,AREA,QUANT,1.00000E+000,Sample Area (cm^2)
7,DENSITY,QUANT,7.87000E+000,Density (g/cm^3)
8,EQUIV,QUANT,2.79200E+001,Equiv. Wt
9,IRCOMP,TOGGLE,T,IR Comp


### MFM data

Provide name of the subdirectory containing the mass flow meter measurement data.

In [26]:
path_mfm = path_raw_data / '03_MFM'

Search directory for further subdirectories and print them.

In [27]:
subdirectories_mfm = {index:directory for index, directory in enumerate(path_mfm.iterdir())}
for index, directory in subdirectories_mfm.items():
    print(f"{index}: {directory.name}")

0: CAD14-Cu@AB


Choose subdirectory by its index.

In [28]:
subdirectory_index_mfm = 0
selected_subdirectory_mfm = subdirectories_mfm[subdirectory_index_mfm]
print(selected_subdirectory_mfm)

/mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/data/Rohdaten/03_MFM/CAD14-Cu@AB


Provide suffix of the file that contains the data.

In [29]:
suffix_mfm = 'csv'

Instantiate the ``MFMParser`` to parse MFM output files and show available files in the selected directory.

In [30]:
mfmparser = MFMParser(selected_subdirectory_mfm, suffix_mfm)
files_dict_mfm = mfmparser.available_files
for index, mfm_file in files_dict_mfm.items():
    print(f"{index}: {mfm_file.name}")

0: Bench-2h-GSS_CAD14-Cu@AB_200_50c_24h.csv
1: Bench-2h-GSS_CAD14-Cu@AB_200_50c_24h_truncated.csv


Chose file to be parsed.

In [31]:
file_index_mfm = 1
file_mfm = files_dict_mfm[file_index_mfm]
file_mfm.name

'Bench-2h-GSS_CAD14-Cu@AB_200_50c_24h_truncated.csv'

Extract the experimental data from it using the ``MFMParser`` and load into the data model.

In [32]:
experimental_data_df_mfm, experimental_data_dict_mfm = mfmparser.extract_exp_data(file_index_mfm)
mfm = lib.Measurement(
            measurement_type=lib.enums.MeasurementType.MFM.value,
            experimental_data=[value for value in experimental_data_dict_mfm.values()],
        )
experiment.measurements.append(mfm)

Print mfm data.

In [33]:
experimental_data_df_mfm

,Datetime,Time,Signal,Flow_rate
0,2023-02-06 09:58:48,0,3258,5.090180
1,2023-02-06 09:58:50,2,3267,5.104674
2,2023-02-06 09:58:52,4,3273,5.114520
3,2023-02-06 09:58:54,6,3278,5.122616
4,2023-02-06 09:58:56,8,3290,5.139893
...,...,...,...,...
78,2023-02-06 10:01:24,156,2472,3.862794
79,2023-02-06 10:01:26,158,2425,3.788688
80,2023-02-06 10:01:28,160,2408,3.762172
81,2023-02-06 10:01:30,162,2425,3.789833


In [34]:
print(experimental_data_dict_mfm.items())

dict_items([('datetime', Data(id='data5', quantity='Date time', values=[datetime.datetime(2023, 2, 6, 9, 58, 48), datetime.datetime(2023, 2, 6, 9, 58, 50), datetime.datetime(2023, 2, 6, 9, 58, 52), datetime.datetime(2023, 2, 6, 9, 58, 54), datetime.datetime(2023, 2, 6, 9, 58, 56), datetime.datetime(2023, 2, 6, 9, 58, 58), datetime.datetime(2023, 2, 6, 9, 59), datetime.datetime(2023, 2, 6, 9, 59, 2), datetime.datetime(2023, 2, 6, 9, 59, 4), datetime.datetime(2023, 2, 6, 9, 59, 6), datetime.datetime(2023, 2, 6, 9, 59, 8), datetime.datetime(2023, 2, 6, 9, 59, 10), datetime.datetime(2023, 2, 6, 9, 59, 12), datetime.datetime(2023, 2, 6, 9, 59, 14), datetime.datetime(2023, 2, 6, 9, 59, 16), datetime.datetime(2023, 2, 6, 9, 59, 18), datetime.datetime(2023, 2, 6, 9, 59, 20), datetime.datetime(2023, 2, 6, 9, 59, 22), datetime.datetime(2023, 2, 6, 9, 59, 24), datetime.datetime(2023, 2, 6, 9, 59, 26), datetime.datetime(2023, 2, 6, 9, 59, 28), datetime.datetime(2023, 2, 6, 9, 59, 30), datetime.dat

In [35]:
# truncated_mfm_experimental_data_df= mfm_experimental_data_df.truncate(after=10)
# truncated_mfm_experimental_data_df

### GC data

Provide name of the subdirectory containing the mass flow meter measurement data.

In [36]:
path_gc = path_raw_data / '02_GC'

Search directory for further subdirectories and print them.

In [37]:
subdirectories_gc = {index:directory for index, directory in enumerate(path_gc.iterdir())}
for index, directory in subdirectories_gc.items():
    print(f"{index}: {directory.name}")

0: CAD14-Cu@AB


Choose subdirectory by its index.

In [38]:
subdirectory_index_gc = 0
selected_subdirectory_gc = subdirectories_gc[subdirectory_index_gc]
print(selected_subdirectory_gc)

/mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/data/Rohdaten/02_GC/CAD14-Cu@AB


Search subdirectory for further subsubdirectories and print them.

In [39]:
subsubdirectories_gc = {index:directory for index, directory in enumerate(selected_subdirectory_gc.iterdir())}
for index, directory in subsubdirectories_gc.items():
    print(f"{index}: {directory.name}")

0: JH-1H 2023-02-06 10-00-18


Choose subsubdirectory by its index.

In [40]:
subsubdirectory_index_gc = 0
selected_subsubdirectory_gc = subsubdirectories_gc[subsubdirectory_index_gc]
print(selected_subsubdirectory_gc)

/mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18


In [41]:
exp_directories_gc = {index:directory for index, directory in enumerate(selected_subsubdirectory_gc.iterdir()) if directory.is_dir()}
for index, directory in exp_directories_gc.items():
    print(f"{index}: {directory.name}")

3: JH_GASPRODUKTE.M
4: JH_GASPRODUKTE_30MIN.M
6: NV-F0101.D
7: NV-F0102.D
8: NV-F0103.D
9: NV-F0104.D
10: NV-F0201.D


Choose dirctories of idividual GC measurements to be used for calculation by their indices.

In [42]:
indices_exp_directories_gc = [7,8,9]
selected_subdirectories_gc = [exp_directories_gc[file] for file in indices_exp_directories_gc]
for sub in selected_subdirectories_gc:  
    print(sub)

/mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D
/mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D
/mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D


Provide filenames of the files that contains the meta data and experimental data, respectively.

In [43]:
filename_exp_gc = 'report01.CSV'
filename_meta_gc = 'report00.CSV'

Initialize GCParser.

In [44]:
gcparser = GCParser(selected_subdirectories_gc, filename_meta_gc, filename_exp_gc)

Show available metadata files contained in the selected directory.

In [45]:
metadata_dict_gc = gcparser.available_meta_files
for index, gc_file in metadata_dict_gc.items():
    print(f"{index}: {gc_file.name}")

0: report00.CSV
1: report00.CSV
2: report00.CSV


Show available experimental data files contained in the selected directory.

In [46]:
exp_data_dict_gc = gcparser.available_exp_files
for index, gc_file in exp_data_dict_gc.items():
    print(f"{index}: {gc_file.name}")

0: report01.CSV
1: report01.CSV
2: report01.CSV


In [47]:
exp_data_dict_gc[0]

PosixPath('/mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/report01.CSV')

In [48]:
import pandas as pd
exp_data_df_gc = pd.read_csv(
    exp_data_dict_gc[0],
    sep=",",
    names=[
        "Peak_number",
        "Retention_time",
        "Signal",
        "Peak_type",
        "Peak_area",
        "Peak_height",
        "Peak_area_percentage",
    ],
    engine="python",
    encoding="utf-16_le",
)
exp_data_df_gc

,Peak_number,Retention_time,Signal,Peak_type,Peak_area,Peak_height,Peak_area_percentage
0,1,1.729967,1,PBAN,69.171577,32.512886,0.098238
1,2,2.909973,1,BBA,65492.746094,3794.478271,93.013605
2,3,3.434230,2,BV,164.157028,43.253098,0.233138
3,4,3.657794,2,VB,141.173935,49.408844,0.200497
4,5,6.045472,2,BB,1624.073730,347.834717,2.306530
5,6,12.997822,1,BB,2876.952637,88.829025,4.085884
6,7,14.194683,2,BB,43.731697,14.139935,0.062108


Select GC files to be parsed.

Metadata

In [49]:
indices_gc_meta = [0,1,2]

Experimental data

In [50]:
indices_gc_exp = [0,1,2]

Extract the metadata and experimental data from them and load into the dataset.

In [51]:

list_df_meta = []
list_df_exp = []
for index in indices_gc_exp:
    metadata_df_gc, metadata_gc= gcparser.extract_metadata(index)
    exp_data_df_gc, exp_data_gc = gcparser.extract_exp_data(index)
    gc = lib.Measurement(
        measurement_type=lib.enums.MeasurementType.GC.value,
        metadata=[value for value in metadata_gc.values()],
        experimental_data=[value for value in exp_data_gc.values()]
    )
    experiment.measurements.append(gc)
    list_df_meta.append(metadata_df_gc)
    list_df_exp.append(exp_data_df_gc)

Print content of first metadata file to check if parsing works properly.

In [52]:
list_df_meta[0]

,parameter,value,description
0,Sample Name,NaN,NaN
1,Sample Info,NaN,NaN
2,Data File,D:\GC\Kurz\CAD14-Cu@AB\JH-1H 2023-02-06 10-00-18\,NV-F0102.D
3,Acq. Instrument,Instrument 1,NaN
4,Analysis Method,D:\GC\Kurz\CAD14-Cu@AB\JH-1H 2023-02-06 10-00-18\,JH_GASPRODUKTE.M
5,Method Info,NaN,NaN
6,Results Created,06.02.2023 10:32:26,NaN
7,Results Created by,MS,NaN
8,Acq. Method,JH_GASPRODUKTE.M,NaN
9,Injection Date,"06-Feb-23, 10:17:24",NaN


Print content of first experimental data file to check.

In [53]:
list_df_exp[0]

,Peak_number,Retention_time,Signal,Peak_type,Peak_area,Peak_height,Peak_area_percentage
0,1,1.729967,1,PBAN,69.171577,32.512886,0.098238
1,2,2.909973,1,BBA,65492.746094,3794.478271,93.013605
2,3,3.434230,2,BV,164.157028,43.253098,0.233138
3,4,3.657794,2,VB,141.173935,49.408844,0.200497
4,5,6.045472,2,BB,1624.073730,347.834717,2.306530
5,6,12.997822,1,BB,2876.952637,88.829025,4.085884
6,7,14.194683,2,BB,43.731697,14.139935,0.062108


In [54]:
# hplc_path = raw_data_path / '04_HPLC'
# pressure_path = raw_data_path / '05_Pressure'

Print current state of experiment object.

In [55]:
print(experiment.json())

{
  "id": "experiment0",
  "measurements": [
    {
      "id": "measurement1",
      "measurement_type": "Potentiostatic Measurement",
      "metadata": [
        {
          "id": "metadata1",
          "parameter": "PSTAT",
          "value": "REF3000-19129",
          "data_type": "PSTAT",
          "description": "Potentiostat"
        },
        {
          "id": "metadata2",
          "parameter": "IINIT",
          "value": "-2.00000E+002",
          "data_type": "QUANT",
          "description": "Initial I (mA/cm^2)"
        },
        {
          "id": "metadata3",
          "parameter": "TINIT",
          "value": "3.60000E+003",
          "data_type": "QUANT",
          "description": "Initial Time (s)"
        },
        {
          "id": "metadata4",
          "parameter": "IFINAL",
          "value": "-2.00000E+002",
          "data_type": "QUANT",
          "description": "Final I (mA/cm^2)"
        },
        {
          "id": "metadata5",
          "parameter": "TFINAL

---
## Analysis
---

In [56]:
analysis = lib.Analysis()

Assign peak areas to species.

The peak areas recorded by the GC have to be matched with the correct species. The individial ``Area`` is selected by its corresponding ``Peak_Number``. It is possible that the same species is accountable for multiple peaks, i.d. multiple peaks are assigned to the same species.


In [57]:
assign_peak_dict={
    'H2': [1],
    'CO2': [2],
    'CO': [6],
    'CH4': [3],
    # 'C2H4': [5],
    # 'C2H6': [4],
}
peak_area_dict = assign_peaks(experiment, assign_peak_dict)

for species, peak_area in peak_area_dict.items():
    print(f"{species}: {peak_area}")

H2: 69.1715774536133
CO2: 65492.74609375
CO: 2876.95263671875
CH4: 164.157028198242


Set calibration input values and import into the data model.

In [58]:
path_calibration_data = path_data / 'calibration'

In [59]:
json_file_path = path_calibration_data / 'calibration.json'

In [63]:
with open(json_file_path, 'r') as file:
    calibration_data = json.load(file)
calibrations = []
for value in calibration_data.values():
    print(value['concentrations'])
#     calibration = lib.Calibration(
#         species=value['species'],
#         peak_area=lib.Data(
#             quantity=lib.enums.Quantity.PEAKAREA.value,
#             unit=None,
#             values=value['peak_areas'],
#         ),
#         concentrations=lib.Data(
#             quantity=lib.enums.Quantity.CONCENTRATION.value,
#             unit='%',
#             values=value['concentrations']
#         )
#     )
#     calibrations.append(calibration)

[5, 10, 20]
[0.5, 1, 5]
[0, 50]
[5, 10]
[0.5, 2, 3]
[0, 5]


Initialize calibrator.

In [ ]:
'hallo'

In [65]:
calibrator = Calibrator.from_json_file(path_to_json=json_file_path)

TypeError: descriptor '__repo__' for 'Calibration' objects doesn't apply to a 'Calibration' object

Load calibration data.

In [ ]:
calibrator.load_data_from_json()

To determine the concentrations of the individual species, a calibration has to be performed in advance to match the individual values for ``Area`` with their corresponding concentrations.

In [ ]:
for species in lib.enums.Species:
    print(species.value)



In [ ]:
calibrations = []
for cali in calibration_input_list:
    calibration = lib.Calibration(
        species=cali['species'],
        peak_area=lib.Data(values=cali["peak_areas"], unit=None),
        concentration=lib.Data(
            values=cali["concentrations"], unit='%'
        ),
    )
    calibrations.append(calibration)

In [ ]:
analysis.calibrations = calibrations
# analysis.calibrations[0].slope.values
experiment.analysis = analysis


Calibrate using the ``calibrate`` method of the ``Calculator`` module.

In [ ]:
calculator=Calculator(experiment=experiment)
experiment= calculator.calibrate()
experiment.analysis.calibrations[0].slope.values
# calibration_df, calibration_dict=calculator.calibrate(calibration_input_dict)
# calibration_df
# for species, value in calibration_dict.items():
#     print(f"{species}: {value}")
#     # print(lib.Calibration(value))

In [ ]:
analysis = lib.Analysis()
analysis.calibrations = [calibration for calibration in calibration_dict.values()]
experiment.analysis = analysis

Print current state of the experiment object.

In [ ]:
print(experiment.json())

Calculate ``volumetric`` ``fractions`` in % out of the peak areas using the determined calibration curve.

In [ ]:
volumetric_fractions_df = calculator.calculate_volumetric_fractions(peak_area_dict=peak_area_dict, calibration_df=calibration_df)
volumetric_fractions_df

Set the ``correction`` ``factors``.

In [ ]:
correction_factors_dict= {
    'H2':1.01,
    'CO':0.74,
    'CO2':1.00,
    'CH4':0.76,
    # 'C2H4':,
    # 'C2H6':,
}

Calculate the ``conversion`` ``factor`` using the correction factors.

In [ ]:
conversion_factor = calculator.calculate_conversion_factor(
    volumetric_fractions_df=volumetric_fractions_df, correction_factors_dict=correction_factors_dict
)
conversion_factor

Get ``volumetric`` ``flow`` ``mean`` in ml/min at the time of the GC measurement.

The mass flow at the time of the GC measurement is determined by matching the time of the gc measurement with the corresponding times of the mass flow measurements. Errors in the mass flows due to strong fluctuations are minimized by calculating the mean by averaging over a certain number (=``radius``) of measuring points before and after the time of the GC measurement. The radius has to be specified in accordance with the strength of fluctuations.

In [ ]:
mean_radius = 10
volumetric_flow_mean = get_volumetric_flow_mean(experiment, mean_radius)
volumetric_flow_mean

Calculate the ``real`` ``volumetric`` ``flow`` in ml/min as a product of the ``volumetric`` ``flow`` ``mean`` and the ``conversion`` ``factor``.

In [ ]:
real_volumetric_flow = volumetric_flow_mean*conversion_factor
real_volumetric_flow

In [ ]:
# vol_flow_real= calculator.calculate_real_volumetric_flow(conversion_factor = conversion_factor, measured_volumetric_flow_mean = vol_flow_mean )
# vol_flow_real

Calculate volumetric flow fractions in %.

In [ ]:
volumetric_flow_fractions_df=calculator.calculate_volumetric_flow_fractions(
    real_volumetric_flow=real_volumetric_flow, volumetric_fractions_df=volumetric_fractions_df
)
volumetric_flow_fractions_df

Calculate material flow in mmol/min.

In [ ]:
material_flow_df = calculator.calcualte_material_flow(volumetric_flow_fractions_df=volumetric_flow_fractions_df)
material_flow_df

Get initial current in mA and initial time in s.

In [ ]:
initial_current, initial_time = get_initial_time_and_current(experiment)
print(f'Initial current in mA: {initial_current}')
print(f'Initial time in s: {initial_time}')

Calculate theoretical material flow in mmol/min.

In [ ]:
electrode_surface_area = 1.0 # cm^2
theoretical_material_flow_df=calculator.calculate_theoretical_material_flow(
    initial_current=initial_current, initial_time=initial_time, electrode_surface_area=electrode_surface_area
)
theoretical_material_flow_df

Calculate Faraday efficiency and load into dataset.

In [ ]:
faraday_efficiency_df = material_flow_df['Material_flow'] / theoretical_material_flow_df['Theoretical_material_flow']
faraday_efficiency_df

---
## DaRUS upload
---

In [ ]:
dataset.experiments.append(experiment)

In [ ]:
with open(json_files[index_dataset], "w") as f:
    f.write(dataset.json())

In [ ]:
button = widgets.Button(description="Append experiment", layout=widgets.Layout(width='30%', height='80px'))
button.style.button_color = 'darkcyan'
button.style.text_color = 'lightgrey'
button.style.font_size = '30px'


output = widgets.Output()

display(button, output)

def click_on_button(b):
    with output:
        print("Experiment successfully appended.")

button.on_click(click_on_button)

In [ ]:
# %%html
# <style>
# .cell-output-ipywidget-background {
#    background-color: transparent !important;
# }
# .cell-output-ipywidget-foreground {


    
# .jp-OutputArea-output {
#    background-color: transparent;
# }  
# </style>

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>